In [51]:
import pandas as pd
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split
from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.multioutput import MultiOutputRegressor
import warnings
warnings.filterwarnings('ignore')

### Helper Functions

In [2]:
def loadData(file):
    data = pd.read_csv(file)
    print('Raw shape: ',data.shape)
    data['Date'] = pd.to_datetime(data.Date)
    print('Days: ',len(set(data.Date)))
    return data

In [3]:
def getTimeSeries(df):
    table = pd.pivot_table(df, values='vehicle_count', index=['Date','Hour'],
                    columns=['DOLocationID'], aggfunc=np.sum, fill_value=0)
    return table

In [4]:
def zscoreNormalizeSpatial(matrix):
    m = matrix.copy()
    for i in range(m.shape[0]):
        m[i, :] = (m[i, :] - m[i, :].mean()) / (m[i, :].std()+1e-10)
        
    return m

In [5]:
def standardize(matrix):
    m = matrix.copy()
    scaler = StandardScaler()
    scaler.fit(m)
    t = scaler.transform(m)
    return scaler, t

In [6]:
def inverse_standardize(matrix, scaler):
    t = matrix.copy()
    return scaler.inverse_transform(t)

In [7]:
def addLag(dataset, maxlag, lagColumns):
    dataset_list = [dataset]

    for l in range(1, maxlag+1):
        df = dataset.shift(l)
        df = df[lagColumns]
        df.columns = [c+'_lag_'+str(l) for c in df.columns]
        dataset_list.append(df)

    dataset = pd.concat(dataset_list, axis=1).dropna()
    return dataset

In [8]:
def get_rmse(matrix1, matrix2):
    sumSquareError = np.mean(np.power(matrix1 - matrix2,2))
    rmse = np.power(sumSquareError,0.5)
    return rmse

In [9]:
def get_weights(rawdata, zontoBorough):
    
    rawdata['Borough'] = rawdata['DOLocationID'].apply(lambda x:zontoBorough[x])
    
    borough_df = rawdata[['vehicle_count','Borough']].groupby(by='Borough').sum().reset_index()

    zone_df = rawdata[['vehicle_count','DOLocationID']].groupby(by='DOLocationID').sum().reset_index()

    zone_df['Borough'] = zone_df['DOLocationID'].apply(lambda x:zontoBorough[x])

    zone_df = pd.merge(borough_df, zone_df, on=['Borough'], how='inner')

    zone_df['zone_weight'] = zone_df.vehicle_count_y / zone_df.vehicle_count_x

    zone_df = zone_df[['Borough', 'DOLocationID', 'zone_weight']]

    return zone_df

#### Preparing Data

In [10]:
hub = 'LGA'
tune_hyp_params = False

In [16]:
dataDir = '/home/mingyi/Dropbox/UrbanTemporalNetworks/processedData/'
file = dataDir + hub + 'VehicleByHour2019fromHub.csv'

In [17]:
rawdata = loadData(file)

Raw shape:  (2295120, 4)
Days:  365


In [18]:
edge_data = getTimeSeries(rawdata)
edge_data = edge_data.reset_index()
edge_data.head(3)

DOLocationID,Date,Hour,1,2,3,4,5,6,7,8,...,256,257,258,259,260,261,262,263,264,265
0,2019-01-01,0,0,0,0,0,0,0,2,0,...,0,1,0,0,2,0,1,6,0,9
1,2019-01-01,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,3
2,2019-01-01,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Community Level Aggregation

In [19]:
zones = pd.read_csv('/home/mingyi/Dropbox/UrbanTemporalNetworks/Data/ZonetoComm.csv')
zones.head(2)

,start_id,start_community
0,1,0.0
1,2,4.2


In [20]:
zones['start_community'] = zones.start_community.astype(str)

zontoComm = dict(zip(zones.start_id.values,zones.start_community.values))

In [49]:
edge_data = edge_data[[i for i in edge_data.columns if i in zones.start_id]]

In [30]:
comm_data = rawdata.copy(deep=True)
comm_data = comm_data.loc[comm_data['DOLocationID'].isin(zones.start_id)]
comm_data['DOLocationID'] = comm_data['DOLocationID'].apply(lambda x:zontoComm[x])
comm_data.head(2)

,Date,DOLocationID,vehicle_count,Hour
0,2019-01-01,0.0,0.0,0
1,2019-01-01,0.0,0.0,1


In [31]:
comm_data = getTimeSeries(comm_data)
comm_data = comm_data.reset_index()
comm_data.head(2)

DOLocationID,Date,Hour,0.0,0.1,0.2,1.0,1.1,1.2,1.3,2.0,...,4.0,4.1,4.2,4.3,4.4,4.5,5.0,5.1,5.2,5.3
0,2019-01-01,0,10,3,15,8,19,0,16,1,...,0,8,6,5,0,8,0,0,0,0
1,2019-01-01,1,0,0,2,2,2,1,1,2,...,0,0,2,5,0,1,0,0,0,0


In [33]:
rawdata = rawdata.loc[rawdata['DOLocationID'].isin(zones.start_id)]
zone_weights = get_weights(rawdata, zontoComm)
zone_weights.head(2)

,Borough,DOLocationID,zone_weight
0,0.0,1,0.004551
1,0.0,48,0.078985


### Merge External Data Features

In [35]:
externalDataDir = "/home/mingyi/Dropbox/UrbanTemporalNetworks/HongData/"+hub+'2019/'
extFile = externalDataDir+"external.csv"

In [36]:
extDf = pd.read_csv(extFile)
print(extDf.shape)
extDf.head(2)

(8760, 14)


,Date,Hour,arrival,PRCP,SNOW,SNWD,TMAX,DOW,Tue,Wed,Thur,Fri,Sat,Sun
0,2019-01-01,0,3.0,0.06,0.0,0.0,58.0,1,1,0,0,0,0,0
1,2019-01-01,1,1.0,0.06,0.0,0.0,58.0,1,1,0,0,0,0,0


In [38]:
selected_columns = ['Date', 'Hour', 'arrival', 'PRCP', 'SNOW', 'SNWD', 'TMAX', 'DOW', 'Tue',
       'Wed', 'Thur', 'Fri', 'Sat', 'Sun']
extDf = extDf[selected_columns]

In [41]:
print(comm_data.shape)
print(extDf.shape)

(8760, 26)
(8760, 14)


In [42]:
comm_data['Date'] = pd.to_datetime(comm_data['Date'])
extDf['Date'] = pd.to_datetime(extDf['Date'])

In [43]:
comm_data = pd.merge(comm_data,extDf, on=['Date', 'Hour'], how='inner')
print(comm_data.shape)
comm_data['Date'] = comm_data['Date'].dt.date
comm_data.head()

(8760, 38)


,Date,Hour,0.0,0.1,0.2,1.0,1.1,1.2,1.3,2.0,...,SNOW,SNWD,TMAX,DOW,Tue,Wed,Thur,Fri,Sat,Sun
0,2019-01-01,0,10,3,15,8,19,0,16,1,...,0.0,0.0,58.0,1,1,0,0,0,0,0
1,2019-01-01,1,0,0,2,2,2,1,1,2,...,0.0,0.0,58.0,1,1,0,0,0,0,0
2,2019-01-01,2,1,0,0,1,0,0,0,1,...,0.0,0.0,58.0,1,1,0,0,0,0,0
3,2019-01-01,3,4,0,0,0,0,0,0,0,...,0.0,0.0,58.0,1,1,0,0,0,0,0
4,2019-01-01,4,6,0,2,0,0,0,3,1,...,0.0,0.0,58.0,1,1,0,0,0,0,0


In [44]:
comm_data.columns

Index(['Date', 'Hour', '0.0', '0.1', '0.2', '1.0', '1.1', '1.2', '1.3', '2.0',
       '2.1', '2.2', '2.3', '3.0', '3.1', '3.2', '4.0', '4.1', '4.2', '4.3',
       '4.4', '4.5', '5.0', '5.1', '5.2', '5.3', 'arrival', 'PRCP', 'SNOW',
       'SNWD', 'TMAX', 'DOW', 'Tue', 'Wed', 'Thur', 'Fri', 'Sat', 'Sun'],
      dtype='object')

In [45]:
lagColumns = ['0.0', '0.1', '0.2', '1.0', '1.1', '1.2', '1.3', '2.0',
       '2.1', '2.2', '2.3', '3.0', '3.1', '3.2', '4.0', '4.1', '4.2', '4.3',
       '4.4', '4.5', '5.0', '5.1', '5.2', '5.3', 'arrival']

DateColumns = ['Date']

targetColumns = ['0.0', '0.1', '0.2', '1.0', '1.1', '1.2', '1.3', '2.0',
       '2.1', '2.2', '2.3', '3.0', '3.1', '3.2', '4.0', '4.1', '4.2', '4.3',
       '4.4', '4.5', '5.0', '5.1', '5.2', '5.3']

In [46]:
maxlag = 12

comm_data_lag = addLag(comm_data, maxlag, lagColumns)

comm_data_lag.shape

(8748, 338)

### Train Test split

In [37]:
# sep = int(0.75*len(comm_data_lag))
# sep

In [53]:
CommR2List = []
EdgeR2List = []
residualDf_list = []
networkPrediction = pd.DataFrame()

for m in range(1,13):
    print()
    print("month: ",m)
    month_index  = pd.to_datetime(comm_data_lag.Date).dt.month == m

    dataset_train = comm_data_lag[~month_index]
    dataset_test = comm_data_lag[month_index]
    print("Train Size: ",dataset_train.shape)
    print("Test Size: ",dataset_test.shape)

    edgeMonthIndex = [False] * maxlag + list(month_index)
    edge_testData = edge_data[edgeMonthIndex]
    select_cols = [c for c in edge_testData.columns if c not in ['Date','Hour']]
    edge_testData = edge_testData[select_cols]
    print("edge test data shape: ",edge_testData.shape)


    X_train = dataset_train.drop(targetColumns+DateColumns , axis = 1)
    X_test = dataset_test.drop(targetColumns+DateColumns , axis = 1)
    y_train = dataset_train[targetColumns]
    y_test = dataset_test[targetColumns]

    rf2 = RandomForestRegressor(random_state = 2019, n_estimators=150, 
                               min_samples_split=3,
                               min_samples_leaf= 2, 
                               max_features= 'sqrt',
                               max_depth= None, 
                               bootstrap= False)

    rf2.fit(X_train,y_train)

    print("Train R2: ",rf2.score(X_train,y_train))
    test_r2 = rf2.score(X_test,y_test)
    print("Test R2: ",test_r2)


    comm_prediction = rf2.predict(X_test)
    edge_prediction_df = pd.DataFrame(comm_prediction)
    edge_prediction_df.columns = y_test.columns

    residual = y_test - comm_prediction
    residual_df = dataset_test[['Date','Hour']]
    residual_df = pd.concat([residual_df,pd.DataFrame(residual)], axis =1)

    boroughs = list(edge_prediction_df.columns)
    for bor in boroughs:
    #     print(bor)

        weight_df = zone_weights[zone_weights.Borough == bor]

    #     print(len(weight_df.DOLocationID))

        for b_zone,z_weight in zip(weight_df.DOLocationID.values,weight_df.zone_weight.values):        
            edge_prediction_df[b_zone] = edge_prediction_df[bor] * z_weight


    select_cols = [c for c in edge_prediction_df.columns if c not in boroughs]
    edge_prediction_df = edge_prediction_df[select_cols]


    edge_prediction_df = edge_prediction_df[edge_testData.columns]
    
    networkPrediction = pd.concat([networkPrediction,edge_prediction_df])
    edge_r2 = r2_score(edge_testData.values, edge_prediction_df.values, multioutput='variance_weighted')
    print("Edge R2: ",edge_r2)

    CommR2List.append(test_r2)
    EdgeR2List.append(edge_r2)
    residualDf_list.append(residual_df)


month:  1
Train Size:  (8016, 338)
Test Size:  (732, 338)
edge test data shape:  (732, 259)
Train R2:  0.9937180061790454
Test R2:  0.870233673314888
Edge R2:  0.6383609843906544

month:  2
Train Size:  (8076, 338)
Test Size:  (672, 338)
edge test data shape:  (672, 259)
Train R2:  0.9936499344854999
Test R2:  0.8909707608876606
Edge R2:  0.6870652712351314

month:  3
Train Size:  (8004, 338)
Test Size:  (744, 338)
edge test data shape:  (744, 259)
Train R2:  0.9935630283464092
Test R2:  0.9196421929691336
Edge R2:  0.7181270951104094

month:  4
Train Size:  (8028, 338)
Test Size:  (720, 338)
edge test data shape:  (720, 259)
Train R2:  0.9936254867920264
Test R2:  0.9056092062191492
Edge R2:  0.6999384996682854

month:  5
Train Size:  (8004, 338)
Test Size:  (744, 338)
edge test data shape:  (744, 259)
Train R2:  0.9935423449676336
Test R2:  0.9040372143296903
Edge R2:  0.7062209626694043

month:  6
Train Size:  (8028, 338)
Test Size:  (720, 338)
edge test data shape:  (720, 259)
Tra

In [54]:
networkPrediction['Date'] = comm_data.iloc[12:]['Date'].values
networkPrediction['Hour'] = comm_data.iloc[12:]['Hour'].values
networkPrediction.to_csv('/home/mingyi/Dropbox/UrbanTemporalNetworks/prediction/%sComm242019.csv'%hub,index=False)

In [55]:
print(np.mean(CommR2List))
print(np.mean(EdgeR2List))

0.8985098973102824
0.6964772530335804


In [64]:
res_df = pd.concat(residualDf_list, axis = 0)
print(res_df.shape)
res_df.head()

(8748, 26)


,Date,Hour,0.0,0.1,0.2,1.0,1.1,1.2,1.3,2.0,...,4.0,4.1,4.2,4.3,4.4,4.5,5.0,5.1,5.2,5.3
12,2019-01-01,12,-15.902222,-3.877778,21.942222,9.188889,25.493333,-0.863333,11.670000,-1.650000,...,-0.291111,-10.422222,-7.462222,-0.040000,1.416667,10.380000,-0.198889,-0.122222,-0.116667,-0.084444
13,2019-01-01,13,-23.573333,-4.076667,5.853333,-2.002222,30.210000,-2.644444,3.361111,-3.233333,...,-0.193333,-3.651111,-11.474444,-8.546667,-0.542222,5.706667,-0.207778,-0.131111,-0.133333,-0.067778
14,2019-01-01,14,-31.161111,15.471111,42.944444,-1.117778,9.070000,-0.208889,12.473333,-4.796667,...,-0.187778,-2.358889,-8.580000,1.975556,0.465556,13.716667,0.778889,-0.196667,-0.190000,-0.107778
15,2019-01-01,15,-58.298889,-3.481111,-7.864444,-3.151111,-1.261111,0.731111,-32.512222,-0.893333,...,-0.153333,-10.186667,-12.717778,-11.070000,0.424444,-6.613333,-0.237778,-0.146667,-0.186667,-0.126667
16,2019-01-01,16,-14.060000,3.791111,67.380000,8.078889,22.204444,-2.555556,-0.776667,-4.227778,...,-0.162222,-7.020000,-13.128889,-11.950000,-0.731111,2.782222,2.767778,-0.211111,-0.190000,-0.127778


In [67]:
res_df.groupby(['Date']).sum().drop(columns='Hour').to_csv('/home/mingyi/Dropbox/DOE_Anomaly_Detection/GMMDataset/selected/RFCV/2019/'+hub+'Comm24RFCVResidDailyAggregated2019.csv')